<a href="https://colab.research.google.com/github/hagabbar/OzGrav_demo/blob/master/OzGrav_VItamin_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!git pull 

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/hagabbar/OzGrav_demo
   a82ae1b..c323a78  master     -> origin/master
Updating a82ae1b..c323a78
Fast-forward
 VICI_code_usage_example.py | 12 +++++++++++-
 1 file changed, 11 insertions(+), 1 deletion(-)


# Set-up

In [1]:
!git clone https://github.com/hagabbar/OzGrav_demo.git

Cloning into 'OzGrav_demo'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 79 (delta 21), reused 57 (delta 14), pack-reused 0
Unpacking objects: 100% (79/79), done.


In [2]:
%cd OzGrav_demo/
%ls

/content/OzGrav_demo
bilby_pe.py         OzGrav_VItamin_demo.ipynb  test_sets/
LICENSE             plotsky.py                 training_sets_3det_9par_256Hz/
make_test_plots.py  plots.py                   VICI_code_usage_example.py
Models/             README.md
Neural_Networks/    requirements.txt


In [0]:
pip install -r requirements.txt

In [0]:
import numpy as np


# Background

This tutorial is based on the paper Bayesian Parameter Estimation using Conditional Variational Autoencoders for Gravitational-wave Astronomy (https://arxiv.org/abs/1909.06296). In this demo, we will show how a form of conditional variational autoencoders work and how to reproduce Bayesian posteriors several orders of magnitude faster than exisitng Bayesian techniques.



## Network Training Portion of VItamin

In [0]:
graph = tf.Graph()
session = tf.Session(graph=graph)
with graph.as_default():

    # PLACE HOLDERS
    bs_ph = tf.placeholder(dtype=tf.int64, name="bs_ph")                       # batch size placeholder
    x_ph = tf.placeholder(dtype=tf.float32, shape=[None, xsh[1]], name="x_ph") # params placeholder
    if n_conv_r1 != None:
        if params['by_channel'] == True:
            y_ph = tf.placeholder(dtype=tf.float32, shape=[None,ysh,len(fixed_vals['det'])], name="y_ph")    # data placeholder
        else:
            y_ph = tf.placeholder(dtype=tf.float32, shape=[None,len(fixed_vals['det']),ysh], name="y_ph")
    else:
        y_ph = tf.placeholder(dtype=tf.float32, shape=[None,ysh], name="y_ph")    # data placeholder
    idx = tf.placeholder(tf.int32)

    # LOAD VICI NEURAL NETWORKS
    r2_xzy = VICI_decoder.VariationalAutoencoder('VICI_decoder', wrap_mask, nowrap_mask, 
                                                  n_input1=z_dimension, n_input2=ysh_conv_r2, n_output=xsh[1], 
                                                  n_weights=n_weights_r2, n_hlayers=n_hlayers_r2, 
                                                  drate=drate, n_filters=n_filters_r2, filter_size=filter_size_r2,
                                                  maxpool=maxpool_r2, n_conv=n_conv_r2, conv_strides=conv_strides_r2, pool_strides=pool_strides_r2, num_det=num_det, batch_norm=batch_norm, by_channel=params['by_channel'], weight_init=params['weight_init'])
    r1_zy = VICI_encoder.VariationalAutoencoder('VICI_encoder', n_input=ysh_conv_r1, n_output=z_dimension, 
                                                  n_weights=n_weights_r1, n_modes=n_modes, 
                                                  n_hlayers=n_hlayers_r1, drate=drate, n_filters=n_filters_r1, 
                                                  filter_size=filter_size_r1,maxpool=maxpool_r1, n_conv=n_conv_r1, conv_strides=conv_strides_r1, pool_strides=pool_strides_r1, num_det=num_det, batch_norm=batch_norm, by_channel=params['by_channel'], weight_init=params['weight_init'])
    q_zxy = VICI_VAE_encoder.VariationalAutoencoder('VICI_VAE_encoder', n_input1=xsh[1], n_input2=ysh_conv_q, 
                                                    n_output=z_dimension, n_weights=n_weights_q, 
                                                    n_hlayers=n_hlayers_q, drate=drate, n_filters=n_filters_q, 
                                                    filter_size=filter_size_q,maxpool=maxpool_q, n_conv=n_conv_q, conv_strides=conv_strides_q, pool_strides=pool_strides_q, num_det=num_det, batch_norm=batch_norm, by_channel=params['by_channel'], weight_init=params['weight_init']) # used to sample from q(z|x,y)?
    tf.set_random_seed(np.random.randint(0,10))

    ramp = (tf.log(tf.dtypes.cast(idx,dtype=tf.float32)) - tf.log(ramp_start))/(tf.log(ramp_end)-tf.log(ramp_start))
    ramp = tf.minimum(tf.math.maximum(0.0,ramp),1.0)
        
    if params['ramp'] == False:
        ramp = 1.0
 
    # reduce the y data size
    y_conv = y_ph

    # GET r1(z|y)
    # run inverse autoencoder to generate mean and logvar of z given y data - these are the parameters for r1(z|y)
    r1_loc, r1_scale, r1_weight = r1_zy._calc_z_mean_and_sigma(y_conv)
    r1_scale = tf.sqrt(SMALL_CONSTANT + tf.exp(r1_scale))
    # get l1 loss term
    l1_loss_weight = ramp*1e-3*tf.reduce_sum(tf.math.abs(r1_weight),1)
    r1_weight = ramp*tf.squeeze(r1_weight)
        
    # define the r1(z|y) mixture model
    bimix_gauss = tfd.MixtureSameFamily(
                      mixture_distribution=tfd.Categorical(logits=ramp*r1_weight),
                      components_distribution=tfd.MultivariateNormalDiag(
                      loc=r1_loc,
                      scale_diag=r1_scale))


    # DRAW FROM r1(z|y) - given the Gaussian parameters generate z samples
    r1_zy_samp = bimix_gauss.sample()        
        
    # GET q(z|x,y)
    q_zxy_mean, q_zxy_log_sig_sq = q_zxy._calc_z_mean_and_sigma(x_ph,y_conv)

    # DRAW FROM q(z|x,y)
    q_zxy_samp = q_zxy._sample_from_gaussian_dist(bs_ph, z_dimension, q_zxy_mean, tf.log(SMALL_CONSTANT + tf.exp(q_zxy_log_sig_sq)))
        
    # GET r2(x|z,y)
    reconstruction_xzy = r2_xzy.calc_reconstruction(q_zxy_samp,y_conv)
    r2_xzy_mean_nowrap = reconstruction_xzy[0]
    r2_xzy_log_sig_sq_nowrap = reconstruction_xzy[1]
    if np.sum(wrap_mask)>0:
        r2_xzy_mean_wrap = reconstruction_xzy[2]
        r2_xzy_log_sig_sq_wrap = reconstruction_xzy[3]

    # COST FROM RECONSTRUCTION - Gaussian parts
    normalising_factor_x = -0.5*tf.log(SMALL_CONSTANT + tf.exp(r2_xzy_log_sig_sq_nowrap)) - 0.5*np.log(2.0*np.pi)   # -0.5*log(sig^2) - 0.5*log(2*pi)
    square_diff_between_mu_and_x = tf.square(r2_xzy_mean_nowrap - tf.boolean_mask(x_ph,nowrap_mask,axis=1))         # (mu - x)^2

    inside_exp_x = -0.5 * tf.divide(square_diff_between_mu_and_x,SMALL_CONSTANT + tf.exp(r2_xzy_log_sig_sq_nowrap)) # -0.5*(mu - x)^2 / sig^2
    reconstr_loss_x = tf.reduce_sum(normalising_factor_x + inside_exp_x,axis=1,keepdims=True)                       # sum_dim(-0.5*log(sig^2) - 0.5*log(2*pi) - 0.5*(mu - x)^2 / sig^2)

    # COST FROM RECONSTRUCTION - Von Mises parts
    if np.sum(wrap_mask)>0:
        con = tf.reshape(tf.math.reciprocal(SMALL_CONSTANT + tf.exp(r2_xzy_log_sig_sq_wrap)),[-1,wrap_len])   # modelling wrapped scale output as log variance
        von_mises = tfp.distributions.VonMises(loc=2.0*np.pi*(tf.reshape(r2_xzy_mean_wrap,[-1,wrap_len])-0.5), concentration=con)   # define p_vm(2*pi*mu,con=1/sig^2)
        reconstr_loss_vm = tf.reduce_sum(von_mises.log_prob(2.0*np.pi*(tf.reshape(tf.boolean_mask(x_ph,wrap_mask,axis=1),[-1,wrap_len]) - 0.5)),axis=1)   # 2pi is the von mises input range
        cost_R = -1.0*tf.reduce_mean(reconstr_loss_x + reconstr_loss_vm) # average over batch
        r2_xzy_mean = tf.gather(tf.concat([r2_xzy_mean_nowrap,r2_xzy_mean_wrap],axis=1),tf.constant(idx_mask),axis=1)
        r2_xzy_scale = tf.gather(tf.concat([r2_xzy_log_sig_sq_nowrap,r2_xzy_log_sig_sq_wrap],axis=1),tf.constant(idx_mask),axis=1) 
    else:
        cost_R = -1.0*tf.reduce_mean(reconstr_loss_x)    
        r2_xzy_mean = r2_xzy_mean_nowrap
        r2_xzy_scale = r2_xzy_log_sig_sq_nowrap

    # compute montecarlo KL - first compute the analytic self entropy of q 
    normalising_factor_kl = -0.5*tf.log(SMALL_CONSTANT + tf.exp(q_zxy_log_sig_sq)) - 0.5*np.log(2.0*np.pi)   # -0.5*log(sig^2) - 0.5*log(2*pi)
    square_diff_between_qz_and_q = tf.square(q_zxy_mean - q_zxy_samp)                                        # (mu - x)^2
    inside_exp_q = -0.5 * tf.divide(square_diff_between_qz_and_q,SMALL_CONSTANT + tf.exp(q_zxy_log_sig_sq))  # -0.5*(mu - x)^2 / sig^2
    log_q_q = tf.reduce_sum(normalising_factor_kl + inside_exp_q,axis=1,keepdims=True)                       # sum_dim(-0.5*log(sig^2) - 0.5*log(2*pi) - 0.5*(mu - x)^2 / sig^2)
    log_r1_q = bimix_gauss.log_prob(q_zxy_samp)   # evaluate the log prob of r1 at the q samples
    KL = tf.reduce_mean(log_q_q - log_r1_q)      # average over batch

    # THE VICI COST FUNCTION
    COST = cost_R + ramp*KL

    # VARIABLES LISTS
    var_list_VICI = [var for var in tf.trainable_variables() if var.name.startswith("VICI")]
        
    # DEFINE OPTIMISER (using ADAM here)
    optimizer = tf.train.AdamOptimizer(params['initial_training_rate']) 
    minimize = optimizer.minimize(COST,var_list = var_list_VICI)
        
    # INITIALISE AND RUN SESSION
    init = tf.global_variables_initializer()
    session.run(init)
    saver = tf.train.Saver()

## Q network
Encoder network which takes as input time series and source parameter truths. Outputs samples from the latent space.


In [0]:
class VariationalAutoencoder(object):

    def __init__(self, name, n_input1=3, n_input2=256, n_output=4, n_weights=2048, n_hlayers=2, drate=0.2, n_filters=8, filter_size=8, maxpool=4, n_conv=2, conv_strides=1, pool_strides=1, num_det=1, batch_norm=False, by_channel=False, weight_init='xavier'):
        
        self.n_input1 = n_input1
        self.n_input2 = n_input2
        self.n_output = n_output
        self.n_weights = n_weights

        self.n_hlayers = n_hlayers
        self.n_conv = n_conv
        self.drate = drate
        self.n_filters = n_filters
        self.filter_size = filter_size
        self.maxpool = maxpool
        self.conv_strides = conv_strides
        self.pool_strides = pool_strides
        self.num_det = num_det
        self.batch_norm = batch_norm
        self.by_channel = by_channel
        self.weight_init = weight_init

        network_weights = self._create_weights()
        self.weights = network_weights
        self.nonlinearity = tf.nn.relu

    def _calc_z_mean_and_sigma(self,x,y):
        with tf.name_scope("VICI_VAE_encoder"):

            # Reshape input to a 3D tensor - single channel
            if self.n_conv is not None:
                if self.by_channel == True:
                    conv_pool = tf.reshape(y, shape=[-1, 1, y.shape[1], self.num_det])
                    for i in range(self.n_conv):
                        weight_name = 'w_conv_' + str(i)
                        bias_name = 'b_conv_' + str(i)
                        conv_pre = tf.add(tf.nn.conv2d(conv_pool, self.weights['VICI_VAE_encoder'][weight_name],strides=[1,1,self.conv_strides[i],1],padding='SAME'),self.weights['VICI_VAE_encoder'][bias_name])
                        conv_post = self.nonlinearity(conv_pre)
                        if self.batch_norm == True:
                            conv_batchNorm = tf.nn.batch_normalization(conv_post,tf.Variable(tf.zeros([1,conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),tf.Variable(tf.ones([1,conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),None,None,0.000001)
                            conv_dropout = tf.layers.dropout(conv_batchNorm,rate=self.drate)
                        else:
                            conv_dropout = tf.layers.dropout(conv_post,rate=self.drate)
                        conv_pool = tf.nn.max_pool(conv_dropout,ksize=[1, 1, self.maxpool[i], 1],strides=[1, 1, self.pool_strides[i], 1],padding='SAME')

                    fc = tf.concat([x,tf.reshape(conv_pool, [-1, int(conv_pool.shape[2]*conv_pool.shape[3])])],axis=1)
                if self.by_channel == False:
                    conv_pool = tf.reshape(y, shape=[-1, y.shape[1], y.shape[2], 1])
                    for i in range(self.n_conv):
                        weight_name = 'w_conv_' + str(i)
                        bias_name = 'b_conv_' + str(i)
                        conv_pre = tf.add(tf.nn.conv2d(conv_pool, self.weights['VICI_VAE_encoder'][weight_name],strides=[1,self.conv_strides[i],self.conv_strides[i],1],padding='SAME'),self.weights['VICI_VAE_encoder'][bias_name])
                        conv_post = self.nonlinearity(conv_pre)
                        if self.batch_norm == True:
                            conv_batchNorm = tf.nn.batch_normalization(conv_post,tf.Variable(tf.zeros([conv_post.shape[1],conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),tf.Variable(tf.ones([conv_post.shape[1],conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),None,None,0.000001)
                        conv_pool = tf.nn.max_pool(conv_batchNorm,ksize=[1, self.maxpool[i], self.maxpool[i], 1],strides=[1, self.pool_strides[i], self.pool_strides[i], 1],padding='SAME')

                    fc = tf.concat([x,tf.reshape(conv_pool, [-1, int(conv_pool.shape[1]*conv_pool.shape[2]*conv_pool.shape[3])])],axis=1)
            else:
                fc = tf.concat([x,y],axis=1)

            hidden_dropout = fc
            for i in range(self.n_hlayers):
                weight_name = 'w_hidden_' + str(i)
                bias_name = 'b_hidden' + str(i)
                hidden_pre = tf.add(tf.matmul(hidden_dropout, self.weights['VICI_VAE_encoder'][weight_name]), self.weights['VICI_VAE_encoder'][bias_name])
                hidden_post = self.nonlinearity(hidden_pre)
                if self.batch_norm == True:
                    hidden_batchNorm = tf.nn.batch_normalization(hidden_post,tf.Variable(tf.zeros([hidden_post.shape[1]], dtype=tf.float32)),tf.Variable(tf.ones([hidden_post.shape[1]], dtype=tf.float32)),None,None,0.000001)
                    hidden_dropout = tf.layers.dropout(hidden_batchNorm,rate=self.drate)
                else:
                    hidden_dropout = tf.layers.dropout(hidden_post,rate=self.drate)
            loc = tf.add(tf.matmul(hidden_dropout, self.weights['VICI_VAE_encoder']['w_loc']), self.weights['VICI_VAE_encoder']['b_loc'])
            scale = tf.add(tf.matmul(hidden_dropout, self.weights['VICI_VAE_encoder']['w_scale']), self.weights['VICI_VAE_encoder']['b_scale'])

            tf.summary.histogram('loc', loc)
            tf.summary.histogram('scale', scale)
            return loc, scale


    def _sample_from_gaussian_dist(self, num_rows, num_cols, mean, log_sigma_sq):
        with tf.name_scope("sample_in_z_space"):
            eps = tf.random_normal([num_rows, num_cols], 0, 1., dtype=tf.float32)
            sample = tf.add(mean, tf.multiply(tf.sqrt(tf.exp(log_sigma_sq)), eps))
        return sample

    def _create_weights(self):
        all_weights = collections.OrderedDict()
        with tf.variable_scope("VICI_VAE_ENC"):
            # Encoder
            all_weights['VICI_VAE_encoder'] = collections.OrderedDict()
            
            if self.n_conv is not None:
                dummy = 1
                for i in range(self.n_conv):
                    weight_name = 'w_conv_' + str(i)
                    bias_name = 'b_conv_' + str(i)
                    # orthogonal init
                    if self.weight_init == 'Orthogonal':
                        shape_init = (self.filter_size[i],dummy*self.n_filters[i])
                        initializer = tf.keras.initializers.Orthogonal()
                        all_weights['VICI_VAE_encoder'][weight_name] = tf.Variable(tf.reshape(initializer(shape=shape_init),[self.filter_size[i], 1, dummy, self.n_filters[i]]), dtype=tf.float32)
                    # Variance scaling
                    if self.weight_init == 'VarianceScaling':
                        shape_init = (self.filter_size[i],dummy*self.n_filters[i])
                        initializer = tf.keras.initializers.VarianceScaling()
                        all_weights['VICI_VAE_encoder'][weight_name] = tf.Variable(tf.reshape(initializer(shape=shape_init),[self.filter_size[i], 1, dummy, self.n_filters[i]]), dtype=tf.float32)
                    # xavier initilization
                    if self.weight_init == 'xavier':
                        all_weights['VICI_VAE_encoder'][weight_name] = tf.Variable(tf.reshape(vae_utils.xavier_init(self.filter_size[i], dummy*self.n_filters[i]),[self.filter_size[i], 1, dummy, self.n_filters[i]]), dtype=tf.float32)
                    all_weights['VICI_VAE_encoder'][bias_name] = tf.Variable(tf.zeros([self.n_filters[i]], dtype=tf.float32))
                    tf.summary.histogram(weight_name, all_weights['VICI_VAE_encoder'][weight_name])
                    tf.summary.histogram(bias_name, all_weights['VICI_VAE_encoder'][bias_name])
                    dummy = self.n_filters[i]

                total_pool_stride_sum = 0
                for j in range(len(self.maxpool)):
                    if self.maxpool[j] != 1 and self.pool_strides[j] != 1:
                        total_pool_stride_sum += 1
                    else:
                        if self.maxpool[j] != 1:
                            total_pool_stride_sum += 1
                        if self.pool_strides[j] != 1:
                            total_pool_stride_sum += 1
                    if self.conv_strides[j] != 1:
                        total_pool_stride_sum += 1
                if self.by_channel == True:
                    fc_input_size = self.n_input1 + int(self.n_input2*self.n_filters[i]/(2**total_pool_stride_sum))
                else:
                    fc_input_size = self.n_input1 + int(self.n_input2*self.n_filters[i]/(2**total_pool_stride_sum)*2) 
            else:
                fc_input_size = self.n_input1 + self.n_input2

            for i in range(self.n_hlayers):
                weight_name = 'w_hidden_' + str(i)
                bias_name = 'b_hidden' + str(i)
                all_weights['VICI_VAE_encoder'][weight_name] = tf.Variable(vae_utils.xavier_init(fc_input_size, self.n_weights[i]), dtype=tf.float32)
                all_weights['VICI_VAE_encoder'][bias_name] = tf.Variable(tf.zeros([self.n_weights[i]], dtype=tf.float32))
                tf.summary.histogram(weight_name, all_weights['VICI_VAE_encoder'][weight_name])
                tf.summary.histogram(bias_name, all_weights['VICI_VAE_encoder'][bias_name])
                fc_input_size = self.n_weights[i]
            all_weights['VICI_VAE_encoder']['w_loc'] = tf.Variable(vae_utils.xavier_init(self.n_weights[-1], self.n_output),dtype=tf.float32)
            all_weights['VICI_VAE_encoder']['b_loc'] = tf.Variable(tf.zeros([self.n_output], dtype=tf.float32), dtype=tf.float32)
            tf.summary.histogram('w_loc', all_weights['VICI_VAE_encoder']['w_loc'])
            tf.summary.histogram('b_loc', all_weights['VICI_VAE_encoder']['b_loc'])
            all_weights['VICI_VAE_encoder']['w_scale'] = tf.Variable(vae_utils.xavier_init(self.n_weights[-1], self.n_output),dtype=tf.float32)
            all_weights['VICI_VAE_encoder']['b_scale'] = tf.Variable(tf.zeros([self.n_output], dtype=tf.float32), dtype=tf.float32)
            tf.summary.histogram('w_scale', all_weights['VICI_VAE_encoder']['w_scale'])
            tf.summary.histogram('b_scale', all_weights['VICI_VAE_encoder']['b_scale'])

            all_weights['prior_param'] = collections.OrderedDict()
        
        return all_weights

## R1 network
Encoder network which takes as input time series only. Outputs latent space samples.

In [0]:
class VariationalAutoencoder(object):

    def __init__(self, name, n_input=256, n_output=4, n_weights=2048, n_modes=2, n_hlayers=2, drate=0.2, n_filters=8, filter_size=8, maxpool=4, n_conv=2, conv_strides=1, pool_strides=1, num_det=1, batch_norm=False, by_channel=False, weight_init='xavier'):
        
        self.n_input = n_input
        self.n_output = n_output
        self.n_weights = n_weights
        self.n_filters = n_filters
        self.filter_size = filter_size
        self.n_hlayers = n_hlayers
        self.n_conv = n_conv
        self.n_modes = n_modes
        self.drate = drate
        self.maxpool = maxpool
        self.conv_strides = conv_strides
        self.pool_strides = pool_strides
        self.num_det = num_det
        self.batch_norm = batch_norm
        self.by_channel = by_channel
        self.weight_init = weight_init

        network_weights = self._create_weights()
        self.weights = network_weights


        self.nonlinearity = tf.nn.relu
        self.nonlinearity_mean = tf.clip_by_value

    def _calc_z_mean_and_sigma(self,x):
        with tf.name_scope("VICI_encoder"):
 
            # Reshape input to a 3D tensor - single channel
            if self.n_conv is not None:
                if self.by_channel == True:
                    conv_pool = tf.reshape(x, shape=[-1, 1, x.shape[1], self.num_det])

                    # network for messing with kernel size
                    for i in range(self.n_conv):
                        weight_name = 'w_conv_' + str(i)
                        bias_name = 'b_conv_' + str(i)
                        conv_pre = tf.add(tf.nn.conv2d(conv_pool, self.weights['VICI_encoder'][weight_name],strides=[1,1,self.conv_strides[i],1],padding='SAME'),self.weights['VICI_encoder'][bias_name])
                        conv_post = self.nonlinearity(conv_pre)
                        if self.batch_norm == True:
                            conv_batchNorm = tf.nn.batch_normalization(conv_post,tf.Variable(tf.zeros([1,conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),tf.Variable(tf.ones([1,conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),None,None,0.000001)
                            conv_dropout = tf.layers.dropout(conv_batchNorm,rate=self.drate)
                        else:
                            conv_dropout = tf.layers.dropout(conv_post,rate=self.drate)
                        conv_pool = tf.nn.max_pool(conv_dropout,ksize=[1, 1, self.maxpool[i], 1],strides=[1, 1, self.pool_strides[i], 1],padding='SAME')
                    
                    fc = tf.reshape(conv_pool, [-1, int(conv_pool.shape[2]*conv_pool.shape[3])])
                if self.by_channel == False:
                    conv_pool = tf.reshape(x, shape=[-1, x.shape[1], x.shape[2], 1])

                    # network for messing with kernel size
                    for i in range(self.n_conv):
                        weight_name = 'w_conv_' + str(i)
                        bias_name = 'b_conv_' + str(i)
                        conv_pre = tf.add(tf.nn.conv2d(conv_pool, self.weights['VICI_encoder'][weight_name],strides=[1,self.conv_strides[i],self.conv_strides[i],1],padding='SAME'),self.weights['VICI_encoder'][bias_name])
                        conv_post = self.nonlinearity(conv_pre)
                        if self.batch_norm == True:
                            conv_batchNorm = tf.nn.batch_normalization(conv_post,tf.Variable(tf.zeros([conv_post.shape[1],conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),tf.Variable(tf.ones([conv_post.shape[1],conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),None,None,0.000001)
                        conv_pool = tf.nn.max_pool(conv_batchNorm,ksize=[1, self.maxpool[i], self.maxpool[i], 1],strides=[1, self.pool_strides[i], self.pool_strides[i], 1],padding='SAME')

                    fc = tf.reshape(conv_pool, [-1, int(conv_pool.shape[1]*conv_pool.shape[2]*conv_pool.shape[3])])
            else:
                fc = x

            hidden_dropout = fc
            for i in range(self.n_hlayers):
                weight_name = 'w_hidden_' + str(i)
                bias_name = 'b_hidden' + str(i)
                hidden_pre = tf.add(tf.matmul(hidden_dropout, self.weights['VICI_encoder'][weight_name]), self.weights['VICI_encoder'][bias_name])
                hidden_post = self.nonlinearity(hidden_pre)
                if self.batch_norm == True:
                    hidden_batchNorm = tf.nn.batch_normalization(hidden_post,tf.Variable(tf.zeros([hidden_post.shape[1]], dtype=tf.float32)),tf.Variable(tf.ones([hidden_post.shape[1]], dtype=tf.float32)),None,None,0.000001)
                    hidden_dropout = tf.layers.dropout(hidden_batchNorm,rate=self.drate)
                else:
                    hidden_dropout = tf.layers.dropout(hidden_post,rate=self.drate)
            loc = tf.add(tf.matmul(hidden_dropout, self.weights['VICI_encoder']['w_loc']), self.weights['VICI_encoder']['b_loc'])
            scale = tf.add(tf.matmul(hidden_dropout, self.weights['VICI_encoder']['w_scale']), self.weights['VICI_encoder']['b_scale'])
            weight = tf.add(tf.matmul(hidden_dropout, self.weights['VICI_encoder']['w_weight']), self.weights['VICI_encoder']['b_weight']) 

            tf.summary.histogram('loc', loc)
            tf.summary.histogram('scale', scale)
            tf.summary.histogram('weight', weight)
            return tf.reshape(loc,(-1,self.n_modes,self.n_output)), tf.reshape(scale,(-1,self.n_modes,self.n_output)), tf.reshape(weight,(-1,self.n_modes))    

    def _create_weights(self):
        all_weights = collections.OrderedDict()
        with tf.variable_scope("VICI_ENC"):            
            all_weights['VICI_encoder'] = collections.OrderedDict()

            if self.n_conv is not None:
                dummy = 1
                for i in range(self.n_conv):
                    weight_name = 'w_conv_' + str(i)
                    bias_name = 'b_conv_' + str(i)
                    # orthogonal init
                    if self.weight_init == 'Orthogonal':
                        shape_init = (self.filter_size[i],dummy*self.n_filters[i])
                        initializer = tf.keras.initializers.Orthogonal()
                        all_weights['VICI_encoder'][weight_name] = tf.Variable(tf.reshape(initializer(shape=shape_init),[self.filter_size[i], 1, dummy, self.n_filters[i]]), dtype=tf.float32)
                    # Variance scaling
                    if self.weight_init == 'VarianceScaling':
                        shape_init = (self.filter_size[i],dummy*self.n_filters[i])
                        initializer = tf.keras.initializers.VarianceScaling()
                        all_weights['VICI_encoder'][weight_name] = tf.Variable(tf.reshape(initializer(shape=shape_init),[self.filter_size[i], 1, dummy, self.n_filters[i]]), dtype=tf.float32)
                    # xavier initilization
                    if self.weight_init == 'xavier':
                        all_weights['VICI_encoder'][weight_name] = tf.Variable(tf.reshape(vae_utils.xavier_init(self.filter_size[i], dummy*self.n_filters[i]),[self.filter_size[i], 1, dummy, self.n_filters[i]]), dtype=tf.float32)
                    all_weights['VICI_encoder'][bias_name] = tf.Variable(tf.zeros([self.n_filters[i]], dtype=tf.float32))
                    tf.summary.histogram(weight_name, all_weights['VICI_encoder'][weight_name])
                    tf.summary.histogram(bias_name, all_weights['VICI_encoder'][bias_name])
                    dummy = self.n_filters[i]

                total_pool_stride_sum = 0
                for j in range(len(self.maxpool)):
                    if self.maxpool[j] != 1 and self.pool_strides[j] != 1:
                        total_pool_stride_sum += 1
                    else:
                        if self.maxpool[j] != 1:
                            total_pool_stride_sum += 1
                        if self.pool_strides[j] != 1:
                            total_pool_stride_sum += 1
                    if self.conv_strides[j] != 1:
                        total_pool_stride_sum += 1
                if self.by_channel == True:
                    fc_input_size = int(self.n_input*self.n_filters[i]/(2**total_pool_stride_sum))
                else:
                    fc_input_size = int((self.n_input*self.n_filters[i]/(2**total_pool_stride_sum)*2))
            else:
                fc_input_size = self.n_input

            for i in range(self.n_hlayers):
                weight_name = 'w_hidden_' + str(i)
                bias_name = 'b_hidden' + str(i)
                all_weights['VICI_encoder'][weight_name] = tf.Variable(vae_utils.xavier_init(fc_input_size, self.n_weights[i]), dtype=tf.float32)
                all_weights['VICI_encoder'][bias_name] = tf.Variable(tf.zeros([self.n_weights[i]], dtype=tf.float32))
                tf.summary.histogram(weight_name, all_weights['VICI_encoder'][weight_name])
                tf.summary.histogram(bias_name, all_weights['VICI_encoder'][bias_name])
                fc_input_size = self.n_weights[i]
            all_weights['VICI_encoder']['w_loc'] = tf.Variable(vae_utils.xavier_init(self.n_weights[-1], self.n_output*self.n_modes),dtype=tf.float32)
            all_weights['VICI_encoder']['b_loc'] = tf.Variable(tf.zeros([self.n_output*self.n_modes], dtype=tf.float32), dtype=tf.float32)
            tf.summary.histogram('w_loc', all_weights['VICI_encoder']['w_loc'])
            tf.summary.histogram('b_loc', all_weights['VICI_encoder']['b_loc'])
            all_weights['VICI_encoder']['w_scale'] = tf.Variable(vae_utils.xavier_init(self.n_weights[-1], self.n_output*self.n_modes),dtype=tf.float32)
            all_weights['VICI_encoder']['b_scale'] = tf.Variable(tf.zeros([self.n_output*self.n_modes], dtype=tf.float32), dtype=tf.float32)
            tf.summary.histogram('w_scale', all_weights['VICI_encoder']['w_scale'])
            tf.summary.histogram('b_scale', all_weights['VICI_encoder']['b_scale'])
            all_weights['VICI_encoder']['w_weight'] = tf.Variable(vae_utils.xavier_init(self.n_weights[-1], self.n_modes),dtype=tf.float32)
            all_weights['VICI_encoder']['b_weight'] = tf.Variable(tf.zeros([self.n_modes], dtype=tf.float32), dtype=tf.float32)
            tf.summary.histogram('w_weight', all_weights['VICI_encoder']['w_weight'])
            tf.summary.histogram('b_weight', all_weights['VICI_encoder']['b_weight'])

            all_weights['prior_param'] = collections.OrderedDict()
        
        return all_weights

## R2 network
Decoder network which takes as input samples from latent space produced by q network. Outputs samples from the posterior when trained properly.

In [0]:
class VariationalAutoencoder(object):

    def __init__(self, name, wrap_mask, nowrap_mask, n_input1=4, n_input2=256, n_output=3, n_weights=2048, n_hlayers=2, drate=0.2, n_filters=8, filter_size=8, maxpool=4, n_conv=2, conv_strides=1, pool_strides=1, num_det=1, batch_norm=False,by_channel=False, weight_init='xavier'):
        
        self.n_input1 = n_input1                    # actually the output size
        self.n_input2 = n_input2                    # actually the output size
        self.n_output = n_output                  # the input data size
        self.n_weights = n_weights                # the number of weights were layer
        self.n_hlayers = n_hlayers
        self.n_conv = n_conv
        self.n_filters = n_filters
        self.filter_size = filter_size
        self.maxpool = maxpool
        self.conv_strides = conv_strides
        self.pool_strides = pool_strides
        self.name = name                          # the name of the network
        self.drate = drate                        # dropout rate
        self.wrap_mask = wrap_mask                # mask identifying wrapped indices
        self.nowrap_mask = nowrap_mask            # mask identifying non-wrapped indices
        self.num_det = num_det
        self.batch_norm = batch_norm
        self.by_channel = by_channel
        self.weight_init = weight_init 

        network_weights = self._create_weights()
        self.weights = network_weights

        self.nonlinear_loc_nowrap = tf.sigmoid    # activation for non-wrapped location params
        self.nonlinear_loc_wrap = tf.sigmoid      # activation for wrapped location params
        self.nonlinear_scale_nowrap = tf.identity # activation for non-wrapped scale params
        self.nonlinear_scale_wrap = tf.nn.relu    # activation for wrapped scale params  
        self.nonlinearity = tf.nn.relu            # activation between hidden layers

    def calc_reconstruction(self, z, y):
        with tf.name_scope("VICI_decoder"):

            # Reshape input to a 3D tensor - single channel
            if self.n_conv is not None:
                if self.by_channel == True:
                    conv_pool = tf.reshape(y, shape=[-1, 1, y.shape[1], self.num_det])
                    for i in range(self.n_conv):            
                        weight_name = 'w_conv_' + str(i)
                        bias_name = 'b_conv_' + str(i)
                        conv_pre = tf.add(tf.nn.conv2d(conv_pool, self.weights['VICI_decoder'][weight_name],strides=[1,1,self.conv_strides[i],1],padding='SAME'),self.weights['VICI_decoder'][bias_name])
                        conv_post = self.nonlinearity(conv_pre)
                        if self.batch_norm == True:
                            conv_batchNorm = tf.nn.batch_normalization(conv_post,tf.Variable(tf.zeros([1,conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),tf.Variable(tf.ones([1,conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),None,None,0.000001)
                            conv_dropout = tf.layers.dropout(conv_batchNorm,rate=self.drate)
                        else:
                            conv_dropout = tf.layers.dropout(conv_post,rate=self.drate)
                        conv_pool = tf.nn.max_pool(conv_dropout,ksize=[1, 1, self.maxpool[i], 1],strides=[1, 1, self.pool_strides[i], 1],padding='SAME')

                    fc = tf.concat([z,tf.reshape(conv_pool, [-1, int(conv_pool.shape[2]*conv_pool.shape[3])])],axis=1)            
                if self.by_channel == False:
                    conv_pool = tf.reshape(y, shape=[-1, y.shape[1], y.shape[2], 1])
                    for i in range(self.n_conv):
                        weight_name = 'w_conv_' + str(i)
                        bias_name = 'b_conv_' + str(i)
                        conv_pre = tf.add(tf.nn.conv2d(conv_pool, self.weights['VICI_decoder'][weight_name],strides=[1,self.conv_strides[i],self.conv_strides[i],1],padding='SAME'),self.weights['VICI_decoder'][bias_name])
                        conv_post = self.nonlinearity(conv_pre)
                        if self.batch_norm == True:
                            conv_batchNorm = tf.nn.batch_normalization(conv_post,tf.Variable(tf.zeros([conv_post.shape[1],conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),tf.Variable(tf.ones([conv_post.shape[1],conv_post.shape[2],conv_post.shape[3]], dtype=tf.float32)),None,None,0.000001)
                        conv_pool = tf.nn.max_pool(conv_batchNorm,ksize=[1, self.maxpool[i], self.maxpool[i], 1],strides=[1, self.pool_strides[i], self.pool_strides[i], 1],padding='SAME')

                    fc = tf.concat([z,tf.reshape(conv_pool, [-1, int(conv_pool.shape[1]*conv_pool.shape[2]*conv_pool.shape[3])])],axis=1)
            else:
                fc = tf.concat([z,y],axis=1)

            hidden_dropout = fc
            for i in range(self.n_hlayers):
                weight_name = 'w_hidden_' + str(i)
                bias_name = 'b_hidden' + str(i)
                hidden_pre = tf.add(tf.matmul(hidden_dropout, self.weights['VICI_decoder'][weight_name]), self.weights['VICI_decoder'][bias_name])
                hidden_post = self.nonlinearity(hidden_pre)
                if self.batch_norm == True:
                    hidden_batchNorm = tf.nn.batch_normalization(hidden_post,tf.Variable(tf.zeros([hidden_post.shape[1]], dtype=tf.float32)),tf.Variable(tf.ones([hidden_post.shape[1]], dtype=tf.float32)),None,None,0.000001)
                    hidden_dropout = tf.layers.dropout(hidden_batchNorm,rate=self.drate)
                else:
                    hidden_dropout = tf.layers.dropout(hidden_post,rate=self.drate)
            loc_all = tf.add(tf.matmul(hidden_dropout, self.weights['VICI_decoder']['w_loc']), self.weights['VICI_decoder']['b_loc'])
            scale_all = tf.add(tf.matmul(hidden_dropout, self.weights['VICI_decoder']['w_scale']), self.weights['VICI_decoder']['b_scale'])

            # split up the output into non-wrapped and wrapped params and apply appropriate activation
            loc_nowrap = self.nonlinear_loc_nowrap(tf.boolean_mask(loc_all,self.nowrap_mask,axis=1))
            scale_nowrap = self.nonlinear_scale_nowrap(tf.boolean_mask(scale_all,self.nowrap_mask,axis=1))
            if np.sum(self.wrap_mask)>0:
                loc_wrap = self.nonlinear_loc_wrap(tf.boolean_mask(loc_all,self.wrap_mask,axis=1))
                scale_wrap = -1.0*self.nonlinear_scale_wrap(tf.boolean_mask(scale_all,self.wrap_mask,axis=1))
                return loc_nowrap, scale_nowrap, loc_wrap, scale_wrap
            else:
                return loc_nowrap, scale_nowrap

    def _create_weights(self):
        all_weights = collections.OrderedDict()

        # Decoder
        with tf.variable_scope("VICI_DEC"):
            all_weights['VICI_decoder'] = collections.OrderedDict()
            
            if self.n_conv is not None:
                dummy = 1
                for i in range(self.n_conv):
                    weight_name = 'w_conv_' + str(i)
                    bias_name = 'b_conv_' + str(i)
                    # orthogonal init
                    if self.weight_init == 'Orthogonal':
                        shape_init = (self.filter_size[i],dummy*self.n_filters[i])
                        initializer = tf.keras.initializers.Orthogonal()
                        all_weights['VICI_decoder'][weight_name] = tf.Variable(tf.reshape(initializer(shape=shape_init),[self.filter_size[i], 1, dummy, self.n_filters[i]]), dtype=tf.float32)
                    # Variance scaling
                    if self.weight_init == 'VarianceScaling':
                        shape_init = (self.filter_size[i],dummy*self.n_filters[i])
                        initializer = tf.keras.initializers.VarianceScaling()
                        all_weights['VICI_decoder'][weight_name] = tf.Variable(tf.reshape(initializer(shape=shape_init),[self.filter_size[i], 1, dummy, self.n_filters[i]]), dtype=tf.float32)
                    # xavier initilization
                    if self.weight_init == 'xavier':
                        all_weights['VICI_decoder'][weight_name] = tf.Variable(tf.reshape(vae_utils.xavier_init(self.filter_size[i], dummy*self.n_filters[i]),[self.filter_size[i], 1, dummy, self.n_filters[i]]), dtype=tf.float32)
                    all_weights['VICI_decoder'][bias_name] = tf.Variable(tf.zeros([self.n_filters[i]], dtype=tf.float32))
                    tf.summary.histogram(weight_name, all_weights['VICI_decoder'][weight_name])
                    tf.summary.histogram(bias_name, all_weights['VICI_decoder'][bias_name])
                    dummy = self.n_filters[i]

                total_pool_stride_sum = 0
                for j in range(len(self.maxpool)):
                    if self.maxpool[j] != 1 and self.pool_strides[j] != 1:
                        total_pool_stride_sum += 1
                    else:
                        if self.maxpool[j] != 1:
                            total_pool_stride_sum += 1
                        if self.pool_strides[j] != 1:
                            total_pool_stride_sum += 1
                    if self.conv_strides[j] != 1:
                        total_pool_stride_sum += 1
                if self.by_channel == True:
                    fc_input_size = self.n_input1 + int(self.n_input2*self.n_filters[i]/(2**total_pool_stride_sum))
                else:
                    fc_input_size = self.n_input1 + int(self.n_input2*self.n_filters[i]/(2**total_pool_stride_sum)*2)
            else:
                fc_input_size = self.n_input1 + self.n_input2

            for i in range(self.n_hlayers):
                weight_name = 'w_hidden_' + str(i)
                bias_name = 'b_hidden' + str(i)
                all_weights['VICI_decoder'][weight_name] = tf.Variable(vae_utils.xavier_init(fc_input_size, self.n_weights[i]), dtype=tf.float32)
                all_weights['VICI_decoder'][bias_name] = tf.Variable(tf.zeros([self.n_weights[i]], dtype=tf.float32))
                tf.summary.histogram(weight_name, all_weights['VICI_decoder'][weight_name])
                tf.summary.histogram(bias_name, all_weights['VICI_decoder'][bias_name])
                fc_input_size = self.n_weights[i]
            all_weights['VICI_decoder']['w_loc'] = tf.Variable(vae_utils.xavier_init(self.n_weights[-1], self.n_output),dtype=tf.float32)
            all_weights['VICI_decoder']['b_loc'] = tf.Variable(tf.zeros([self.n_output], dtype=tf.float32), dtype=tf.float32)
            tf.summary.histogram('w_loc', all_weights['VICI_decoder']['w_loc'])
            tf.summary.histogram('b_loc', all_weights['VICI_decoder']['b_loc'])
            all_weights['VICI_decoder']['w_scale'] = tf.Variable(vae_utils.xavier_init(self.n_weights[-1], self.n_output),dtype=tf.float32)
            all_weights['VICI_decoder']['b_scale'] = tf.Variable(tf.zeros([self.n_output], dtype=tf.float32), dtype=tf.float32)
            tf.summary.histogram('w_scale', all_weights['VICI_decoder']['w_scale'])
            tf.summary.histogram('b_scale', all_weights['VICI_decoder']['b_scale'])
            
            all_weights['prior_param'] = collections.OrderedDict()
        
        return all_weights

# Running VItamin on your own

## Generate Bilby Posterior Test Samples

### Define bounds and default fixed values for source parameters and search

In [0]:
# Source parameter values to use if chosen to be fixed
fixed_vals = {'mass_1':50.0,
        'mass_2':50.0,
        'mc':None,
        'geocent_time':0.0,
        'phase':0.0,
        'ra':1.375,
        'dec':-1.2108,
        'psi':0.0,
        'theta_jn':0.0,
        'luminosity_distance':2000.0,
        'a_1':0.0,
        'a_2':0.0,
	'tilt_1':0.0,
	'tilt_2':0.0,
        'phi_12':0.0,
        'phi_jl':0.0,
        'det':['H1','L1','V1']}                                                 # feel free to edit this if more or less detectors wanted


# Prior bounds on source parameters
bounds = {'mass_1_min':35.0, 'mass_1_max':80.0,
        'mass_2_min':35.0, 'mass_2_max':80.0,
        'M_min':70.0, 'M_max':160.0,
        'geocent_time_min':0.15,'geocent_time_max':0.35,
        'phase_min':0.0, 'phase_max':2.0*np.pi,
        'ra_min':0.0, 'ra_max':2.0*np.pi,
        'dec_min':-0.5*np.pi, 'dec_max':0.5*np.pi,
        'psi_min':0.0, 'psi_max':2.0*np.pi,
        'theta_jn_min':0.0, 'theta_jn_max':np.pi,
        'a_1_min':0.0, 'a_1_max':0.0,
        'a_2_min':0.0, 'a_2_max':0.0,
        'tilt_1_min':0.0, 'tilt_1_max':0.0,
        'tilt_2_min':0.0, 'tilt_2_max':0.0,
        'phi_12_min':0.0, 'phi_12_max':0.0,
        'phi_jl_min':0.0, 'phi_jl_max':0.0,
        'luminosity_distance_min':1000.0, 'luminosity_distance_max':3000.0}

### Define other run parameters

In [0]:
##########################
# Main tunable variables
##########################
ndata = 256                                                                     # sampling frequency
rand_pars = ['mass_1','mass_2','luminosity_distance','geocent_time','phase',
                 'theta_jn','psi','ra','dec']                                   # parameters to randomize (those not listed here are fixed otherwise)
inf_pars=['luminosity_distance','geocent_time','ra','dec'],                     # parameters to infer
batch_size = 64                                                                 # Number training samples shown to neural network per iteration
weight_init = 'xavier',                                                         #[xavier,VarianceScaling,Orthogonal] # Network model weight initialization    
n_modes=7,                                                                      # number of modes in Gaussian mixture model (ideal 7, but may go higher/lower)
initial_training_rate=1e-4,                                                     # initial training rate for ADAM optimiser inference model (inverse reconstruction)
batch_norm=True,                                                                # if true, do batch normalization in all layers of neural network

# FYI, each item in lists below correspond to each layer in networks (i.e. first item first layer)
# pool size and pool stride should be same number in each layer
n_filters_r1 = [33, 33, 33],                                                    # number of convolutional filters to use in r1 network (must be divisible by 3)
n_filters_r2 = [33, 33, 33],                                                    # number of convolutional filters to use in r2 network (must be divisible by 3)
n_filters_q = [33, 33, 33],                                                     # number of convolutional filters to use in q network  (must be divisible by 3)
filter_size_r1 = [7,7,7],                                                       # size of convolutional fitlers in r1 network
filter_size_r2 = [7,7,7],                                                       # size of convolutional filters in r2 network
filter_size_q = [7,7,7],                                                        # size of convolutional filters in q network
drate = 0.5,                                                                    # dropout rate to use in fully-connected layers
maxpool_r1 = [1,2,1],                                                           # size of maxpooling to use in r1 network
conv_strides_r1 = [1,1,1],                                                      # size of convolutional stride to use in r1 network
pool_strides_r1 = [1,2,1],                                                      # size of max pool stride to use in r1 network
maxpool_r2 = [1,2,1],                                                           # size of max pooling to use in r2 network
conv_strides_r2 = [1,1,1],                                                      # size of convolutional stride in r2 network
pool_strides_r2 = [1,2,1],                                                      # size of max pool stride in r2 network
maxpool_q = [1,2,1],                                                            # size of max pooling to use in q network
conv_strides_q = [1,1,1],                                                       # size of convolutional stride to use in q network
pool_strides_q = [1,2,1],                                                       # size of max pool stride to use in q network
n_fc = 512                                                                      # Number of neurons in fully-connected layers
z_dimension=8,                                                                  # number of latent space dimensions of model 
n_weights_r1 = [n_fc,n_fc],                                                     # number fully-connected layers of encoders and decoders in the r1 model (inverse reconstruction)
n_weights_r2 = [n_fc,n_fc],                                                     # number fully-connected layers of encoders and decoders in the r2 model (inverse reconstruction)
n_weights_q = [n_fc,n_fc],                                                      # number fully-connected layers of encoders and decoders q model
##########################
# Main tunable variables
##########################

#############################
# optional tunable variables
#############################
run_label = 'ozgrav-demo_%ddet_%dpar_%dHz_run1' % (len(fixed_vals['det']),len(rand_pars),ndata) # label of run
bilby_results_label = 'ozgrav-demo'                                             # label given to bilby results directory
r = 2                                                                           # number (to the power of 2) of test samples to use for testing. r = 2 means you want to use 2^2 (i.e 4) test samples
pe_test_num = 256                                                               # total number of test samples available to use in directory
tot_dataset_size = int(1e4)                                                     # total number of training samples available to use
tset_split = int(1e3)                                                           # number of training samples in each training data file
save_interval = int(5e4)                                                        # number of iterations to save model and plot validation results corner plots
ref_geocent_time=1126259642.5                                                   # reference gps time (not advised to change this)
load_chunk_size = 1e4                                                           # Number of training samples to load in at a time.
samplers=['vitamin','dynesty'],                                                 # Bayesian samplers to use when comparing ML results (vitamin is ML approach) dynesty,ptemcee,cpnest,emcee

# Directory variables
plot_dir="results/%s" % run_label,  # output directory to save results plots
train_set_dir='training_sets_%ddet_%dpar_%dHz/tset_tot-%d_split-%d' % (len(fixed_vals['det']),len(rand_pars),ndata,tot_dataset_size,tset_split), # location of training set
test_set_dir='test_sets/%s/four_parameter_case/test_waveforms' % bilby_results_label,                                                            # location of test set directory waveforms
pe_dir='test_sets/%s/four_parameter_case/test' % bilby_results_label,                                                                            # location of test set directory Bayesian PE samples
#############################
# optional tunable variables
#############################

In [0]:

# Function for getting list of parameters that need to be fed into the models
def get_params():

    # Define dictionary to store values used in rest of code 
    params = dict(
        make_corner_plots = True,                                               # if True, make corner plots
        make_kl_plot = True,                                                    # If True, go through kl plotting function
        make_pp_plot = True,                                                    # If True, go through pp plotting function
        make_loss_plot = False,                                                 # If True, generate loss plot from previous plot data
        Make_sky_plot=False,                                                    # If True, generate sky plots on corner plots
        hyperparam_optim = False,                                               # optimize hyperparameters for model during training using gaussian process minimization
        resume_training=False,                                                  # if True, resume training of a model from saved checkpoint
        load_by_chunks = True,                                                  # if True, load training samples by a predefined chunk size rather than all at once
        ramp = True,                                                            # if true, apply linear ramp to KL loss
        print_values=True,                                                      # optionally print loss values every report interval
        by_channel = True,                                                      # if True, do convolutions as seperate 1-D channels, if False, stack training samples as 2-D images (n_detectors,(duration*sampling_frequency))
        load_plot_data=False,                                                   # Plotting data which has already been generated
        doPE = True,                                                            # if True then do bilby PE when generating new testing samples (not advised to change this)
        gpu_num=0,                                                              # gpu number run is running on
        ndata = ndata,                                                          
        run_label=run_label,                                                    
        bilby_results_label=bilby_results_label,                                
        tot_dataset_size = tot_dataset_size,                                    
        tset_split = tset_split,                                                
        plot_dir=plot_dir,

        # Gaussian Process automated hyperparameter tunning variables
        hyperparam_optim_stop = int(1.5e6),                                     # stopping iteration of hyperparameter optimizer per call (ideally 1.5 million) 
        hyperparam_n_call = 30,                                                 # number of hyperparameter optimization calls (ideally 30)
        load_chunk_size = load_chunk_size,                                      
        load_iteration = int((load_chunk_size * 25)/batch_size),                # How often to load another chunk of training samples
        weight_init = weight_init[0],                                           
        n_samples = 1000,                                                       # number of posterior samples to save per reconstruction upon inference (default 3000) 
        num_iterations=int(1e7)+1,                                              # total number of iterations before ending training of model
        initial_training_rate=initial_training_rate[0],                         
        batch_size=batch_size,                                                  
        batch_norm=batch_norm,                                                  
        report_interval=500,                                                    # interval at which to save objective function values and optionally print info during training
        n_modes=n_modes[0],                                                     

        # FYI, each item in lists below correspond to each layer in networks (i.e. first item first layer)
        # pool size and pool stride should be same number in each layer
        n_filters_r1 = n_filters_r1[0],                                         
        n_filters_r2 = n_filters_r2[0],                                         
        n_filters_q = n_filters_q[0],                                           
        filter_size_r1 = filter_size_r1[0],                                     
        filter_size_r2 = filter_size_r2[0],                                     
        filter_size_q = filter_size_q[0],                                       
        drate = drate[0],                                                       
        maxpool_r1 = maxpool_r1[0],                                             
        conv_strides_r1 = conv_strides_r1[0],                                   
        pool_strides_r1 = pool_strides_r1[0],                                   
        maxpool_r2 = maxpool_r2[0],                                             
        conv_strides_r2 = conv_strides_r2[0],                                   
        pool_strides_r2 = pool_strides_r2[0],                                   
        maxpool_q = maxpool_q[0],                                               
        conv_strides_q = conv_strides_q[0],                                     
        pool_strides_q = pool_strides_q[0],                                     
        ramp_start = 1e4,                                                       # starting iteration of KL divergence ramp (if using)
        ramp_end = 1e5,                                                         # ending iteration of KL divergence ramp (if using)
        save_interval=save_interval,                                            
        plot_interval=save_interval,                                            
        z_dimension=z_dimension[0],                                              
        n_weights_r1 = n_weights_r1[0],                                         
        n_weights_r2 = n_weights_r2[0],                                         
        n_weights_q = n_weights_q[0],                                           
        duration = 1.0,                                                         # length of training/validation/test sample time series in seconds (haven't tried using at any other value than 1s)
        r = r,                                                                  
        rand_pars=rand_pars,                                                    
        corner_parnames = ['m_{1}\,(\mathrm{M}_{\odot})','m_{2}\,(\mathrm{M}_{\odot})','d_{\mathrm{L}}\,(\mathrm{Mpc})','t_{0}\,(\mathrm{seconds})','{\phi}','\Theta_{jn}\,(\mathrm{rad})','{\psi}',r'{\alpha}\,(\mathrm{rad})','{\delta}\,(\mathrm{rad})'], # latex source parameter labels for plotting
        cornercorner_parnames = ['$m_{1}\,(\mathrm{M}_{\odot})$','$m_{2}\,(\mathrm{M}_{\odot})$','$d_{\mathrm{L}}\,(\mathrm{Mpc})$','$t_{0}\,(\mathrm{seconds})$','${\phi}$','$\Theta_{jn}\,(\mathrm{rad})$','${\psi}$',r'${\alpha}\,(\mathrm{rad})$','${\delta}\,(\mathrm{rad})$'], # latex source parameter labels for plotting
        ref_geocent_time=ref_geocent_time,                                      
        training_data_seed=43,                                                  # tensorflow training random seed number
        testing_data_seed=44,                                                   # tensorflow testing random seed number
        wrap_pars=['phase','psi','ra'],                                         # Parameters to apply Von Mises wrapping on (not advised to change) 
        inf_pars=inf_pars,                                                      
        train_set_dir=train_set_dir,
        test_set_dir=test_set_dir,
        pe_dir=pe_dir,
        KL_cycles = 1,                                                          # number of cycles to repeat for the KL approximation
        samplers=samplers,                                                      
    )
    return params


# Save training/test parameters of run
params=get_params()
params['plot_dir']=params['plot_dir'][0]
params['train_set_dir']=params['train_set_dir'][0]
params['test_set_dir']=params['test_set_dir'][0]
params['pe_dir']=params['pe_dir'][0]
params['inf_pars']=params['inf_pars'][0]
params['samplers']=params['samplers'][0]
f = open("params_%s.txt" % params['run_label'],"w")
f.write( str(params) )
f.close()
f = open("params_%s_bounds.txt" % params['run_label'],"w")
f.write( str(bounds) )
f.close()
f = open("params_%s_fixed_vals.txt" % params['run_label'],"w")
f.write( str(fixed_vals) )
f.close()

### Generate Requested number of testing samples

Feel free to use the below command in your own time to generate more testing data

In [0]:
!python VICI_code_usage_example.py --gen_test True --params_file params_ozgrav-demo_3det_9par_256Hz_run1.txt --params_file_bounds params_ozgrav-demo_3det_9par_256Hz_run1_bounds.txt --params_file_fixed_vals params_ozgrav-demo_3det_9par_256Hz_run1_fixed_vals.txt 

## Generate Traning Samples

In [0]:
!python VICI_code_usage_example.py --gen_train True --params_file params_ozgrav-demo_3det_9par_256Hz_run1.txt --params_file_bounds params_ozgrav-demo_3det_9par_256Hz_run1_bounds.txt --params_file_fixed_vals params_ozgrav-demo_3det_9par_256Hz_run1_fixed_vals.txt 

## Train Model

### Train

In [0]:
!python VICI_code_usage_example.py --train True --params_file params_ozgrav-demo_3det_9par_256Hz_run1.txt --params_file_bounds params_ozgrav-demo_3det_9par_256Hz_run1_bounds.txt --params_file_fixed_vals params_ozgrav-demo_3det_9par_256Hz_run1_fixed_vals.txt 

## Compare Bilby vs. VItamin

In [0]:
!python VICI_code_usage_example.py --test True --params_file params_ozgrav-demo_3det_9par_256Hz_run1.txt --params_file_bounds params_ozgrav-demo_3det_9par_256Hz_run1_bounds.txt --params_file_fixed_vals params_ozgrav-demo_3det_9par_256Hz_run1_fixed_vals.txt 

# Further Reading